In [154]:
import numpy as np
import pandas as pd
import math

In [156]:
df=pd.read_csv("TD1.csv", sep=";")
df["Cours de Natixis"] = df["Cours de Natixis"].astype(str).str.replace(",", ".").astype(float)

In [158]:
df.head()

,Date,Cours de Natixis
0,02/01/2015,5.621
1,05/01/2015,5.424
2,06/01/2015,5.329
3,07/01/2015,5.224
4,08/01/2015,5.453


In [159]:
df.tail()

,Date,Cours de Natixis
1018,21/12/2018,4.045
1019,24/12/2018,4.010
1020,27/12/2018,3.938
1021,28/12/2018,4.088
1022,31/12/2018,4.119


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              1023 non-null   object 
 1   Cours de Natixis  1023 non-null   float64
dtypes: float64(1), object(1)
memory usage: 16.1+ KB


In [161]:
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)


## Question A ##

In [163]:
P = df["Cours de Natixis"].values 
returns = (P[1:] - P[:-1]) / P[:-1]        #we calculate the daily returns using current price and previous price
returns = np.append(returns, np.nan)

df["Return"] = returns

df_date = df[(df["Date"] >= "2015-01-01") & (df["Date"] <= "2016-12-31")]    #we take only the data returns between january 2015 and december 2016

df_sorted = df_date.sort_values(by = "Return", ascending = True)    #we sort the returns in increasing order

df_sorted.head()



,Date,Cours de Natixis,Return
377,2016-06-23,4.150,-0.171325
344,2016-05-09,4.386,-0.069083
357,2016-05-26,4.752,-0.061448
404,2016-08-01,3.643,-0.060664
276,2016-02-01,4.470,-0.058613


In [164]:
alpha = 0.05  # probability level here we choosed 5%, just need to change it here to change it for other VaR

### Historical VaR ###

In [166]:
returns = df_sorted["Return"]

returns_sorted=np.sort(returns)
n=len(returns_sorted)

# Find the index corresponding to the value of alpha. With a length n then the index is n*alpha. And so it will give the 95th percentile smallest value
index_empirical = int(alpha * (n-1))   #we take n-1 and not n because indexes start at 0

# Empirical VaR
VaR_empirical = returns_sorted[index_empirical]

print(f"Empirical VaR at {alpha*100}%: {VaR_empirical:}")

Empirical VaR at 5.0%: -0.038933559035124875


### Biweight VaR ###

In [168]:
# we first start with calculating the bandwidth, first we start by finding the standard deviation
expectation = sum(returns)/n
expecation_square=sum(returns**2)/n
std=(expecation_square- expectation*expectation)**(1/2)

h=((4*(std**5))/(3*n))**(1/5)

print("the calculated bandwidth is : ", h)

# now next step is to create a list of values that will be the potential VaR selected
#so we create 10000 points going from minimum return to maximum returns and in those number will be the VaR we select using the Kernel method
x_search = np.linspace(returns.min(), returns.max(), 10000)

cdf = []

for x in x_search :
    #we know that in the kernel formula u = (x - Xi) / h
    #so let's calculate u
    u=(x-returns.values)/h
    
    #we start by putting only 0s in the kernel array. Then the values will change if u is between -1 and 1 or sup 1 and stay at 0 if u inf -1.
    kernel=np.zeros_like(u)
    #in the function we are given that the indicatrice is for |x|<1 (so x>= -1 and and x <= 1 (here x is u)
    #first condition -1 <= u <= 1
    indicatrice= (u>=-1)&(u<=1)
    #so we consider only the value of x that are contained in that interval
    val= u[indicatrice]

    #now let's calculate the integral of K
    # (1-x²)² = 1 - 2x² + x**4
    #  Integral from - 1 to u (-1 because of the indicatrice and u because we calculate the cdf of K(u)
    # so we can decompose in integral from -1 to 0 + integral from 0 to u. And since it is a cdf of a symetric function around 0 then integral from -1 to 0 = 0.5
    # so integral from 0 to u is equal to u - 2/3 u**3 +1/5 u**5
    kernel[indicatrice] = 0.5 + (15/16) * (val - (2/3)*val**3 + (1/5)*val**5)

    #second condition u >1 then the cdf of K will be the integral on all the density function since it will be from -1 to u with indicatrice 
    #saying that u<1 so it become from - 1 to 1 and so the integral on all the density is equal to 1 by definition
    kernel[u > 1] = 1.0
    #now we take the average of the differente values of the kernel (1/n sum(kernel))
    estimation = sum(kernel)/len(kernel)
    cdf.append(estimation)
    
#we convert it to an array so we can substract alpha to each values of the cdf we calculated
cdf = np.array(cdf)
#we find the best argument
idx_var = np.argmin(np.abs(cdf - alpha))
#and finally we use this argument to find which value generated was the best kernel VaR
VaR_kernel = x_search[idx_var]

print("Biweight Kernel VaR at alpha : ", alpha," VaR_kernel = ", VaR_kernel)

the calculated bandwidth is :  0.007250001157290293
Biweight Kernel VaR at alpha :  0.05  VaR_kernel =  -0.03811674623640021


### b) Porportion of returns that exceeded between January 2017 and December 2018 ###

In [208]:
df_newdate=df[ (df["Date"]<="2018-12-31") & (df["Date"] >= "2017-01-01")]  #we make a new dataset with this time the returns 
#between January 2017 and December 2018

In [210]:
#so we sort it again
df_sorted2=df_newdate.sort_values(by = "Return", ascending=True) 
returns2=df_sorted2["Return"]    
returns2=returns2.dropna()
returns2_sorted=np.sort(returns2)

In [212]:
exceptions = returns2<=VaR_kernel   #we check for the values that are below the Kernel VaR calculated
n_exceptions=sum(exceptions)    #we count the how many there was
proportion=n_exceptions/len(returns2_sorted)          #we divided by the number of total returns in the time desired
print("number of days : ",len(returns2_sorted))
print("exceptions : ", n_exceptions)
print("the proportion that exceeded the VaR of the Kernel", proportion)

number of days :  509
exceptions :  8
the proportion that exceeded the VaR of the Kernel 0.015717092337917484


A proportion of 0.0157 mean that 1.57% of the values were below the value. This means that  in the following years (january 2017 to december 2018) the natixis stocks had less spikes in the losses, since we compared with the 5% worst case considering January 2015 to December 2016. We can note that the market for natixis was less volatil and risky in 2017/2018.
The probability we aimed to get was 5% hence we didn't get the result we expected as 1.57% is pretty far from 5%. The model overestimate the risk. It is a bit too safe and conservative

And now let's do the same but with empirical VaR to see

In [215]:
exceptions = returns2<=VaR_empirical   #we check for the values that are below the Kernel VaR calculated
n_exceptions=sum(exceptions)    #we count the how many there was
proportion=n_exceptions/len(returns2_sorted)          #we divided by the number of total returns in the time desired
print("number of days : ",len(returns2_sorted))
print("exceptions : ", n_exceptions)
print("the proportion that exceeded the VaR of the Kernel", proportion)

number of days :  509
exceptions :  8
the proportion that exceeded the VaR of the Kernel 0.015717092337917484


And so apparently we got the same result even tho the empirical VaR was equal to -0.0389 and so was lower so could have expected maybe one or less exceptions

## Question B ##

### Expected Shortfall ###

In [229]:
exceptions_kernel = df_sorted["Return"][df["Return"]<=VaR_kernel]

Expected_Shortfall_kernel=sum(exceptions_kernel)/len(exceptions_kernel)
print(Expected_Shortfall_kernel)

-0.053361818150455334


In [231]:
exceptions_empirical = df_sorted["Return"][df["Return"]<=VaR_empirical]

Expected_Shortfall_empirical=sum(exceptions_empirical)/len(exceptions_empirical)
print(Expected_Shortfall_empirical)

-0.053361818150455334


The expected shortfall is always lower (or equal in some special cases) than the VaR because it is the average of the returns below the VaR treshold. So the average of smaller values will always be smaller than the initial treshold